# Missing and default values

In [8]:
import datajoint as dj
import random
import faker
fake = faker.Faker()

In [9]:
schema = dj.schema('dimitri_default')

In [10]:
fake.profile()

{'job': 'Engineer, communications',
 'company': 'Rodgers Inc',
 'ssn': '556-83-1710',
 'residence': '2975 Contreras Green\nSouth Karen, ID 49364',
 'current_location': (Decimal('-42.4585165'), Decimal('102.313448')),
 'blood_group': 'AB+',
 'website': ['http://www.rogers.com/', 'https://delgado.com/'],
 'username': 'elizabeth34',
 'name': 'Angela Garcia',
 'sex': 'F',
 'address': '5607 Tapia Cliff\nCrystalside, NY 36851',
 'mail': 'gboyd@gmail.com',
 'birthdate': datetime.date(1946, 11, 28)}

In [18]:
@schema
class Person(dj.Manual):
    definition = """
    person_id : int
    ---
    blood_group = "unknown"  : enum('A+', 'A-', 'AB+', 'AB-', 'B+', 'B-', 'O+', 'O-', "unknown")
    name : varchar(60)
    """

In [24]:
Person.insert1(dict(person_id=3, **fake.profile()), ignore_extra_fields=True)

In [25]:
Person()

person_id,blood_group,name
1,A-,Randy Arnold
2,O+,Scott Montgomery
3,AB-,Ronald Reyes


In [26]:
Person.insert1(dict(person_id=5, name="heywood"))

In [27]:
Person()

person_id,blood_group,name
1,A-,Randy Arnold
2,O+,Scott Montgomery
3,AB-,Ronald Reyes
5,unknown,heywood


In [34]:
print(dj.conn().query("SHOW CREATE TABLE dimitri_default.person").fetchone()[1])

CREATE TABLE `person` (
  `person_id` int(11) NOT NULL,
  `blood_group` enum('A+','A-','AB+','AB-','B+','B-','O+','O-','unknown') NOT NULL DEFAULT 'unknown',
  `name` varchar(60) NOT NULL,
  PRIMARY KEY (`person_id`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1


In [35]:
import pymysql 

In [37]:
conn = pymysql.connect(user='dimitri', passwd=dj.config['database.password'], host='db.ust-db.link')

In [39]:
cursor = conn.cursor()

In [43]:
cursor.execute("""
CREATE TABLE dimitri_default.person2 (
    person_id int NOT NULL,
    blood_group enum('A+','A-','AB+','AB-','B+','B-','O+','O-', 'unknown') 
        NOT NULL DEFAULT 'unknown',
    PRIMARY KEY(person_id)
)
""")

0

In [45]:
cursor.execute("""
INSERT dimitri_default.person2 (person_id, blood_group) VALUES (1, DEFAULT)
""")

1

In [46]:
cursor.execute("""
INSERT dimitri_default.person2 (person_id, blood_group) VALUES (2, "O+")
""")

1

In [47]:
cursor.execute("""
INSERT dimitri_default.person2 (person_id, blood_group) VALUES (3, NULL)
""")

IntegrityError: (1048, "Column 'blood_group' cannot be null")

In [48]:
Person()

person_id,blood_group,name
1,A-,Randy Arnold
2,O+,Scott Montgomery
3,AB-,Ronald Reyes
5,unknown,heywood


In [49]:
@schema
class Department(dj.Lookup):
    definition = """
    dept_code  :  char(8)
    ---
    dept_name : varchar(30)
    """
    
    contents = [
        ('BIOL', "Biology"),
        ('MATH', "Mathematics")    
    ]

In [50]:
@schema
class Student(dj.Manual):
    definition = """
    -> Person
    ---
    -> [nullable] Department
    """
    

```sql
    CREATE TABLE student (
        person_id  int NOT NULL,
        dept_code char(8),
        PRIMARY KEY (person_id),
        FOREIGN KEY (person_id) REFERENCES person(person_id),
        FOREIGN KEY (dept_code) REFERENCES department(dept_code)  
    )
```

In [51]:
Person()

person_id,blood_group,name
1,A-,Randy Arnold
2,O+,Scott Montgomery
3,AB-,Ronald Reyes
5,unknown,heywood


In [52]:
Student.insert1((1, "BIOL"))

In [53]:
Student.insert1((2, "MATH"))

In [54]:
Student.insert1((3, None))

In [55]:
Student()

person_id,dept_code
3,None
1,BIOL
2,MATH


In [ ]:
@schema
class Student(dj.Manual):
    definition = """
    -> Person
    """

In [ ]:
@schema
class Major(dj.Manual):
    definition = """
    -> Students
    ---
    -> Department
    """